In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import load_model
import numpy as np
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [ ]:
#Завантаження потрібної моделі у форматі назва.keras
CNN_Spec = load_model("")

In [ ]:
# шлях до справжніх даних
real_path = ""
# шлях до синтезованих даних
fake_path = ""
real = []
fake = []


# Всі досліджені формати аудіо, вибирається відповідно до завантаженої моделі

# Raw audio

# def get_data(folder_path):
#     raw = []
#     for filename in os.listdir(folder_path):
#         audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
#         raw.append(audio)
#     return raw

# Raw audio 0.5

# def get_data(folder_path):
#     raw = []
#     for filename in os.listdir(folder_path):
#         audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
#         audio_parts = np.array_split(audio, 4)
#         for part in audio_parts:
#             raw.append(part)
#     return raw

# Spectrogram

# def get_data(folder_path):
#     specs = []
#     for filename in os.listdir(folder_path):
#         audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
#         spectrogram = abs(librosa.stft(y=audio))
#         # spectrogram = librosa.amplitude_to_db(abs(spectrogram), ref=np.max)
#         specs.append(spectrogram)
#     return specs

# Spectrogram 0.5

# def get_data(folder_path):
#     specs = []    
#     for filename in os.listdir(folder_path):
#         audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
#         audio_parts = np.array_split(audio, 4)
#         for part in audio_parts:
#             spectrogram = abs(librosa.stft(y=part))
#             spectrogram = librosa.amplitude_to_db(spectrogram, ref=np.max)
#             specs.append(spectrogram)
#     return specs

# Mel-spectrogram

# def get_data(folder_path):
#     mel_specs = []
#     for filename in os.listdir(folder_path):
#         audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
#         mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr)
#         # mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
#         mel_specs.append(mel_spec)
#     return mel_specs

# Mel-spectrogram 0.5

# def get_data(folder_path):
#     mel_specs = []
#     for filename in os.listdir(folder_path):
#         audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
#         audio_parts = np.array_split(audio, 4)
#         for part in audio_parts:
#             mel_spec = librosa.feature.melspectrogram(y=part, sr=sr)
#             mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
#             mel_specs.append(mel_spec)
#     return mel_specs

# MFFC

# def get_data(folder_path):
#     mfccs = []
#     for filename in os.listdir(folder_path):
#         audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
#         mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=128)
#         mfccs.append(mfcc)
#     return mfccs

# MFFC 0.5

# def get_data(folder_path):
#     mfccs = []
#     for filename in os.listdir(folder_path):
#         audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
#         audio_parts = np.array_split(audio, 4)
#         for part in audio_parts:
#             mfcc = librosa.feature.mfcc(y=part, sr=sr, n_mfcc=128)
#             mfccs.append(mfcc)
#     return mfccs

real = get_data(real_path)
fake = get_data(fake_path)


In [ ]:
testR=np.array(real)
testF=np.array(fake)

In [ ]:
testR = testR[..., np.newaxis]
testF = testF[..., np.newaxis]

In [ ]:
print(len(real))

In [ ]:
predR = CNN_Spec.predict(testR)
predF = CNN_Spec.predict(testF)

In [ ]:
threshold = 0.5

binary_predictionsR = [1 if pr < threshold else 0 for pr in predR]

real_count = sum(binary_predictionsR)

binary_predictionsF = [1 if pr > threshold else 0 for pr in predF]

fake_count = sum(binary_predictionsF)

print("Кількість справжніх:", real_count, "з", len(predR))
print("Кількість фальшивих:", fake_count, "з", len(predF))
acc = (real_count + fake_count) / (len(real)+len(fake))
acc = "{:.2f}".format(acc*100)

In [ ]:
from sklearn.metrics import confusion_matrix
real_true_labels = [0] * len(real)
fake_true_labels = [1] * len(fake)

all_predicted_labels = np.concatenate((predR, predF))
all_predicted_labels = np.where(all_predicted_labels < 0.5, 0, 1)
all_true_labels = real_true_labels + fake_true_labels

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)
print(conf_matrix)

In [ ]:
import seaborn as sns
sns.heatmap(conf_matrix, 
            annot=True,
            fmt='g', 
            xticklabels=['Real','Fake'],
            yticklabels=['Real','Fake'])
plt.xlabel('Prediction',fontsize=13)
plt.ylabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.show()